In [57]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

import os
os.sys.path
import cv2

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt

In [68]:
#training data (processed)

rawData = np.load("kaggle_dataset_preprocessed/train_images_subset-processed-cropped45x45.npy", encoding="latin1")

X_train = []
# for i in range (0,400):
for i in range (0,len(rawData)):
    X_train.append(rawData[i][1])
    img_pre = X_train[i].reshape(45,45).astype('float32')
    median = cv2.medianBlur(img_pre,3)
    median2 = cv2.medianBlur(median,3)
    median3 = cv2.medianBlur(median2,3)

X_train = np.asarray(X_train)   
X_train = np.vstack(X_train)
X_train = preprocessing.normalize(X_train)

labelDF = pd.read_csv("kaggle_dataset_preprocessed/train_labels_subset.csv", index_col='Id')
label = labelDF['Category'].tolist()
y_train = label[:len(label)]
# y_train = label[:400]
y_train = np.asarray(y_train)    

In [69]:
#validation data (processed)
rawData1 = np.load("kaggle_dataset_preprocessed/valid_images-processed-cropped45x45.npy", encoding="latin1")

X_Valid = []
for i in range (0,len(rawData1)):
# for i in range (0,100):
    X_Valid.append(rawData1[i][1])
    img_pre = X_Valid[i].reshape(45,45).astype('float32')
    median = cv2.medianBlur(img_pre,3)
    median2 = cv2.medianBlur(median,3)
    median3 = cv2.medianBlur(median2,3)

X_Valid = np.asarray(X_Valid)   
X_Valid = np.vstack(X_Valid)
X_Valid = preprocessing.normalize(X_Valid)

labelDF1 = pd.read_csv("kaggle_dataset_preprocessed/valid_labels.csv", index_col='Id')
label1 = labelDF1['Category'].tolist()
# y_valid = label[:100]
y_valid = label[:len(label1)]
y_valid = np.asarray(y_valid)    

In [70]:
print ("len(X_train) = ",len(X_train))
print ("y_train.shape = ",y_train.shape)
print ("len(X_Valid) = ",len(X_Valid))
print ("y_valid.shape = ",y_valid.shape)

len(X_train) =  8000
y_train.shape =  (8000,)
len(X_Valid) =  2000
y_valid.shape =  (2000,)


In [71]:
def eval_classifier(classifier):
    
    classifier.fit(X_train,y_train)
    train_y_pred =classifier.predict(X_train)
    val_y_pred = classifier.predict(X_Valid)
    train_score = accuracy_score(y_train,train_y_pred)
    val_score = accuracy_score(y_valid,val_y_pred)
    print(type(classifier))
    print(f"Train Acc: {train_score}")
    print(f"Val Acc: {val_score}")
    print("\n")

'   \n    try:\n        classifier.fit(X_train,y_train)\n        train_y_pred =classifier.predict(X_train)\n        val_y_pred = classifier.predict(X_Valid)\n       \n    except:\n        classifier.fit(X_train.toarray(), y_train)\n        train_y =classifier.predict(X_train.toarray())\n        val_y = classifier.predict(X_Valid.toarray())        \n        \n    train_score = accuracy_score(y_train,train_y_pred)\n    val_score = accuracy_score(y_valid,val_y_pred)\n\n'

In [72]:
def eval_classifier_quick(classifier):    
    
    classifier.fit(X_train,y_train)
    val_y_pred = classifier.predict(X_Valid)
    val_acc = accuracy_score(y_valid,val_y_pred)
    return val_acc


def val_classifier(classifier):    
    classifier.fit(X_train,y_train)
    val_y_pred = classifier.predict(X_Valid)
    val_acc = accuracy_score(y_valid,val_y_pred)    
    return val_acc
    
def test_classifier_train(classifier):    
    classifier.fit(X_train,y_train)
    train_y_pred = classifier.predict(X_train)
    train_score = accuracy_score(y_train,train_y_pred)    
    return train_score

'    \n    try:\n        classifier.fit(X_train,y_train)\n        train_y_pred = classifier.predict(X_train) \n    except:\n        classifier.fit(X_train.toarray(), y_train)\n        train_y =classifier.predict(X_train.toarray())\n        \n    train_score = accuracy_score(y_train,train_y_pred)    \n    return train_score\n'

In [73]:
svm_params = ParameterGrid({'random_state':[None, 42],'loss':['hinge','squared_hinge'],'C':[0.001,0.01,0.5,1.0,2.0]})
logreg_param = ParameterGrid({'C':[0.001,0.01,0.1,1,10,100]})
KNN_neighbours = ParameterGrid({'n_neighbors':[1,3,5,7,9]})

classifiers= [(svm.LinearSVC, svm_params), (LogisticRegression, logreg_param), (KNeighborsClassifier, KNN_neighbours)]

In [74]:
# find best params for a classifier 
def optimize_parameters(classifier, param_grid):
    best_score=0 #score on validation
    best_params=None
    for params in param_grid:
        print(f"Trying: {params}")
        val_score = eval_classifier_quick(classifier(**params))
        train_score = test_classifier_train(classifier(**params))
        print(f"Val Acc Score: {val_score}\n")
        print(f"Train Acc Score: {train_score}\n")
        if val_score>best_score:
            best_score = val_score
            best_params = params       
            
    print(f"Best params: {best_params}")
    print(f"Best Accuracy Score: {best_score}\n")
    return classifier(**best_params)

In [75]:
for pair in classifiers: # cycle through the classifiers and parameters
    classifier = pair[0]
    param_grid = pair[1]
    print(classifier)    
    best_classifier = optimize_parameters(classifier, param_grid)
    print(f"Val score for best params: {val_classifier(best_classifier)}\n")
    print(f"Train score for best params: {test_classifier_train(best_classifier)}\n")

<class 'sklearn.svm.classes.LinearSVC'>
Trying: {'C': 0.001, 'loss': 'hinge', 'random_state': None}
Val Acc Score: 0.0465

Train Acc Score: 0.38925

Trying: {'C': 0.001, 'loss': 'hinge', 'random_state': 42}
Val Acc Score: 0.0465

Train Acc Score: 0.389

Trying: {'C': 0.001, 'loss': 'squared_hinge', 'random_state': None}
Val Acc Score: 0.051

Train Acc Score: 0.325125

Trying: {'C': 0.001, 'loss': 'squared_hinge', 'random_state': 42}
Val Acc Score: 0.051

Train Acc Score: 0.325125

Trying: {'C': 0.01, 'loss': 'hinge', 'random_state': None}
Val Acc Score: 0.0425

Train Acc Score: 0.478125

Trying: {'C': 0.01, 'loss': 'hinge', 'random_state': 42}
Val Acc Score: 0.0425

Train Acc Score: 0.478

Trying: {'C': 0.01, 'loss': 'squared_hinge', 'random_state': None}
Val Acc Score: 0.0435

Train Acc Score: 0.45925

Trying: {'C': 0.01, 'loss': 'squared_hinge', 'random_state': 42}
Val Acc Score: 0.0435

Train Acc Score: 0.45925

Trying: {'C': 0.5, 'loss': 'hinge', 'random_state': None}
Val Acc Score